In [21]:
import numpy as np
import pandas as pd
from collections import  Counter
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn import datasets
from sklearn import linear_model

In [22]:
transplant_list = pd.read_excel("SimData_Transplant_Ver2.0.xls", index_col = False, header=0)
# wait_list = pd.read_excel("SimData_WList_Ver1.0.xls", index_col = False, header=0)
transplant_list.shape

(4000, 15)

In [23]:
#choose adult
adult_transplant_list = transplant_list[transplant_list['Age_Tx'] >= 18] 

#lives longer than 3 days
adult_transplant_list_clean = adult_transplant_list[adult_transplant_list["Time"] >= 3]

In [24]:
le = LabelEncoder()

In [25]:
adult_transplant_list_clean["Gender"] = le.fit_transform(adult_transplant_list_clean["Gender"])
adult_transplant_list_clean["Bloodtype"] = le.fit_transform(adult_transplant_list_clean["Bloodtype"])
adult_transplant_list_clean["Donor_Bloodtype"] = le.fit_transform(adult_transplant_list_clean["Donor_Bloodtype"])
adult_transplant_list_clean["Inpt_attx"] = le.fit_transform(adult_transplant_list_clean["Inpt_attx"])
adult_transplant_list_clean["Donor_Sex"] = le.fit_transform(adult_transplant_list_clean["Donor_Sex"])
adult_transplant_list_clean["Donor_DCD"] = le.fit_transform(adult_transplant_list_clean["Donor_DCD"])

<ipython-input-25-b3d8edfa3670>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adult_transplant_list_clean["Gender"] = le.fit_transform(adult_transplant_list_clean["Gender"])
<ipython-input-25-b3d8edfa3670>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adult_transplant_list_clean["Bloodtype"] = le.fit_transform(adult_transplant_list_clean["Bloodtype"])
<ipython-input-25-b3d8edfa3670>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [26]:
ATTRS = ['Gender', 'Bloodtype', 'Inpt_attx', 'Donor_Sex', 'Donor_Bloodtype', \
    'Donor_DCD', 'Waittime', 'Age_Tx', 'Donor_Age', 'TransplantBMI', 'Donor_BMI', \
    'MELD', 'MELDNA']

In [27]:
Data = adult_transplant_list_clean[ATTRS]

In [28]:
Target = adult_transplant_list_clean['Time']

In [29]:
Data.shape

(3992, 13)

In [30]:
Target.shape

(3992,)

In [31]:
X_train,X_test,y_train,y_test=train_test_split(Data,Target,test_size=0.25,random_state=0) 

In [32]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, f_regression, f_classif, mutual_info_regression
import time

def grid_search(pipeline, parameters, X, y):
    gs = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=1)
    
    print('Performing grid search...')
    print('pipeline:', [name for name, _ in pipeline.steps])
    print('parameters:')
    print(parameters)
    t0 = time.time()
    gs.fit(X, y)
    print('done in %0.3fs' % (time.time() - t0))
    print()
    
    # print out best 5 results
    mean_score = gs.cv_results_['mean_test_score']
    param_set = gs.cv_results_['params']
    for i in mean_score.argsort()[-5:]:
        print(param_set[i])
        print(gs.cv_results_['mean_test_score'][i])
        print('='*30)
    
    return gs

In [33]:
pipeline = Pipeline(
    [
        ('kbest', SelectKBest()),
        # ('reg', SGDRegressor(max_iter=1000))
        ('reg', linear_model.Lasso(alpha=0.1)),
        # ('reg', linear_model.ElasticNet(random_state=0))
    ]
)

parameters={
    'kbest__score_func': (f_regression, mutual_info_regression),
    'kbest__k':(5, 10, 'all'),
}


In [34]:
gs = grid_search(pipeline, parameters, Data, Target)

Performing grid search...
pipeline: ['kbest', 'reg']
parameters:
{'kbest__score_func': (<function f_regression at 0x7fc2c6c0e430>, <function mutual_info_regression at 0x7fc2c6c1bc10>), 'kbest__k': (5, 10, 'all')}
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    4.4s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    5.4s finished


done in 6.468s

{'kbest__k': 10, 'kbest__score_func': <function f_regression at 0x7fc2c6c0e430>}
-0.002467703776766572
{'kbest__k': 'all', 'kbest__score_func': <function f_regression at 0x7fc2c6c0e430>}
-0.0020742935673350392
{'kbest__k': 'all', 'kbest__score_func': <function mutual_info_regression at 0x7fc2c6c1bc10>}
-0.0020742935673350392
{'kbest__k': 10, 'kbest__score_func': <function mutual_info_regression at 0x7fc2c6c1bc10>}
-0.0017295808421973647
{'kbest__k': 5, 'kbest__score_func': <function mutual_info_regression at 0x7fc2c6c1bc10>}
0.0012953868253361422


In [35]:
from sklearn import linear_model


pipeline = Pipeline(
    [
        ('kbest', SelectKBest()),
        # ('reg', SGDRegressor(max_iter=1000))
        # ('reg', linear_model.Lasso(alpha=0.1)),
        ('reg', linear_model.ElasticNet(random_state=0))
    ]
)

In [36]:
gs = grid_search(pipeline, parameters, Data, Target)

Performing grid search...
pipeline: ['kbest', 'reg']
parameters:
{'kbest__score_func': (<function f_regression at 0x7fc2c6c0e430>, <function mutual_info_regression at 0x7fc2c6c1bc10>), 'kbest__k': (5, 10, 'all')}
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    3.7s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    4.7s finished


done in 5.698s

{'kbest__k': 10, 'kbest__score_func': <function mutual_info_regression at 0x7fc2c6c1bc10>}
-0.001188501431706701
{'kbest__k': 10, 'kbest__score_func': <function f_regression at 0x7fc2c6c0e430>}
-0.0008659378232052317
{'kbest__k': 'all', 'kbest__score_func': <function f_regression at 0x7fc2c6c0e430>}
-0.00043816284422861786
{'kbest__k': 'all', 'kbest__score_func': <function mutual_info_regression at 0x7fc2c6c1bc10>}
-0.00043816284422861786
{'kbest__k': 5, 'kbest__score_func': <function mutual_info_regression at 0x7fc2c6c1bc10>}
0.0019631871915913868


In [37]:
gs = grid_search(pipeline, parameters, X_train, y_train)

Performing grid search...
pipeline: ['kbest', 'reg']
parameters:
{'kbest__score_func': (<function f_regression at 0x7fc2c6c0e430>, <function mutual_info_regression at 0x7fc2c6c1bc10>), 'kbest__k': (5, 10, 'all')}
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    2.6s remaining:    0.8s


done in 3.344s

{'kbest__k': 5, 'kbest__score_func': <function f_regression at 0x7fc2c6c0e430>}
0.00039591450829687955
{'kbest__k': 10, 'kbest__score_func': <function mutual_info_regression at 0x7fc2c6c1bc10>}
0.001083728027764197
{'kbest__k': 10, 'kbest__score_func': <function f_regression at 0x7fc2c6c0e430>}
0.0013596998808903082
{'kbest__k': 'all', 'kbest__score_func': <function f_regression at 0x7fc2c6c0e430>}
0.001598356531466827
{'kbest__k': 'all', 'kbest__score_func': <function mutual_info_regression at 0x7fc2c6c1bc10>}
0.001598356531466827


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    3.3s finished
